# Reducing Failure-Inducing Inputs

A standard problem in debugging is this: Your program fails after processing some large input. Only a _part_ of this input, however, is responsible for the failure. _Reducing_ the input to a failure-inducing minimum not only eases debugging – it also helps in understanding why and when the program fails. In this chapter, we present techniques that _automatically reduce and simplify failure-inducing inputs to a minimum_, notably the popular _Delta Debugging_ technique.

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("6fmJ5l257bM")

**Prerequisites**

* Using the "delta debugging" technique for reduction has no specific prerequisites.
* To understand the `DeltaDebugger` implementation, reading [the chapter on tracing](Tracer.ipynb) is recommended.

This chapter is adapted from [a similar chapter in "The Fuzzing Book"](https://www.fuzzingbook.org/html/Reducer.html). The material has been adapted to be independent from the `fuzzingbook` infrastructure, to build on general delta debugging (`dd`), and to provide a simpler invocation interface.

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.DeltaDebugger import <identifier>
```

and then make use of the following features.


A _reducer_ takes a failure-inducing input and reduces it to the minimum that still reproduces the failure.  This chapter provides a `DeltaDebugger` class that implements such a reducer.

Here is a simple example: An arithmetic expression causes an error in the Python interpreter:

```python
>>> def myeval(inp: str) -> Any:
>>>     return eval(inp)
>>> with ExpectError(ZeroDivisionError):
>>>     myeval('1 + 2 * 3 / 0')
```
Can we reduce this input to a minimum? _Delta Debugging_ is a simple and robust reduction algorithm. We provide a `DeltaDebugger` class that is used in conjunction with a (failing) function call:

```python
with DeltaDebugger() as dd:
    fun(args...)
dd
```

The class automatically determines minimal arguments that cause the function to fail with the same exception as the original. Printing out the class object reveals the minimized call.

```python
>>> with DeltaDebugger() as dd:
>>>     myeval('1 + 2 * 3 / 0')
>>> dd
```
The input is reduced to the minimum: We get the essence of the division by zero.

There also is an interface to access the reduced input(s) programmatically. The method `min_args()` returns a dictionary in which all function arguments are minimized:

```python
>>> dd.min_args()
```
In contrast, `max_args()` returns a dictionary in which all function arguments are maximized, but still pass:

```python
>>> dd.max_args()
```
The method `min_arg_diff()` returns a triple of 
* passing input,
* failing input, and
* their minimal failure-inducing difference:

```python
>>> dd.min_arg_diff()
```
And you can also access the function itself, as well as its original arguments.

```python
>>> dd.function().__name__, dd.args()
```
`DeltaDebugger` processes (i.e., minimizes or maximizes) all arguments that support a `len()` operation and that can be indexed – notably _strings_ and _lists_. If a function has multiple arguments, all arguments that can be processed will be processed.

This chapter also provides a number of superclasses to `DeltaDebugger`, notably `CallCollector`, which obtains the first function call for `DeltaDebugger`. `CallReducer` classes allow for implementing alternate call reduction strategies.



## Why Reducing?

A common problem in debugging is that given an input, only a _small part of that input may be responsible for the failure_. A central part of debugging is to _identify_ these parts – and to simplify (or _reduce_) the input to a minimal form that reproduces the failure – but does and contains as little else as possible.

Here's an example of such a situation.  We have a `mystery()` method that – given its code – can occasionally fail.  But under which circumstances does this actually happen?  We have deliberately obscured the exact condition in order to make this non-obvious.

In [ ]:
import bookutils

In [ ]:
from bookutils import quiz